In [1]:
from simpleicp import PointCloud, SimpleICP
import numpy as np
import torch as torch
import torch.nn.functional as F
import sys
sys.path.append("../dcp-master")
from data import ModelNet40
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import k3d
import trimesh
from pathlib import Path
from util import npmat2euler
from tqdm import tqdm

def visualize_pointcloud(point_cloud1, point_cloud2, point_size, flip_axes=False, name='point_cloud', R=None, t=None):
    plot = k3d.plot(name=name, grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    # if flip_axes:
    #     point_cloud[:, 2] = point_cloud[:, 2] * -1
    #     point_cloud[:, [0, 1, 2]] = point_cloud[:, [0, 2, 1]]
    # t_broadcast = np.broadcast_to(t[:, np.newaxis], (3, point_cloud1.X.shape[0]))
    # point_cloud1 = (R @ point_cloud1.X.T + t_broadcast).T
    plt_points1 = k3d.points(positions=point_cloud1.X.astype(np.float32), point_size=point_size, color=0xd0d0d0)
    plt_points2 = k3d.points(positions=point_cloud2.astype(np.float32), point_size=point_size, color=0x0dd00d)
    plot += plt_points1
    plot += plt_points2
    plt_points1.shader = '3d'
    plt_points2.shader = '3d'
    plot.display()

def transform(point_cloud, R=None, t=None):
    t_broadcast = np.broadcast_to(t[:, np.newaxis], (3, point_cloud.shape[0]))
    return (R @ point_cloud.T + t_broadcast).T

def test_one_epoch(pc):
    mse_ab = 0
    mae_ab = 0
    mse_ba = 0
    mae_ba = 0

    total_loss = 0
    total_cycle_loss = 0
    num_examples = 0
    rotations_ab = []
    translations_ab = []
    rotations_ab_pred = []
    translations_ab_pred = []

    rotations_ba = []
    translations_ba = []
    rotations_ba_pred = []
    translations_ba_pred = []

    eulers_ab = []
    eulers_ba = []
    for i in tqdm(range(len(pc))):
        
        num_examples += 1
        src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba, _ = pc[i]
        rotation_ab = torch.tensor(rotation_ab).double()
        translation_ab = torch.tensor(translation_ab).double()
        rotation_ba = torch.tensor(rotation_ba).double()
        translation_ba = torch.tensor(translation_ba).double()
        euler_ab = torch.tensor(euler_ab).double()
        euler_ba = torch.tensor(euler_ba).double()


        src = PointCloud(src.T, columns=["x", "y", "z"])
        target = PointCloud(target.T, columns=["x", "y", "z"])
        # Create simpleICP object, add point clouds, and run algorithm!
        icp = SimpleICP()
        icp.add_point_clouds(target, src)
        H, X_mov_transformed, rigid_body_transformation_params = icp.run(max_overlap_distance=1)
        X_mov_transformed = torch.tensor(X_mov_transformed)

        # get the predicted rotations and translations
        rotation_ab_pred = H[:3, :3]
        translation_ab_pred = H[:3, 3]
        rotation_ab_pred = torch.tensor(rotation_ab_pred).double()
        translation_ab_pred = torch.tensor(translation_ab_pred).double()
        rotation_ba_pred = rotation_ab_pred.transpose(1, 0).contiguous()
        translation_ba_pred = -torch.matmul(rotation_ba_pred, translation_ab_pred.unsqueeze(1)).squeeze(1)

        ## save rotation and translation
        rotations_ab.append(rotation_ab.unsqueeze(0).detach().cpu().numpy())
        translations_ab.append(translation_ab.unsqueeze(0).detach().cpu().numpy())
        rotations_ab_pred.append(rotation_ab_pred.unsqueeze(0).detach().cpu().numpy())
        translations_ab_pred.append(translation_ab_pred.unsqueeze(0).detach().cpu().numpy())
        eulers_ab.append(euler_ab.unsqueeze(0).numpy())
        ##
        rotations_ba.append(rotation_ba.unsqueeze(0).detach().cpu().numpy())
        translations_ba.append(translation_ba.unsqueeze(0).detach().cpu().numpy())
        rotations_ba_pred.append(rotation_ba_pred.unsqueeze(0).detach().cpu().numpy())
        translations_ba_pred.append(translation_ba_pred.unsqueeze(0).detach().cpu().numpy())
        eulers_ba.append(euler_ba.unsqueeze(0).numpy())
        ##############################################
        identity = torch.eye(3).repeat(1, 1)
        loss = F.mse_loss(torch.matmul(rotation_ab_pred.transpose(1, 0), rotation_ab), identity) \
                + F.mse_loss(translation_ab_pred, translation_ab)
        total_loss += loss.item()
        mse_ab += torch.mean((X_mov_transformed - torch.tensor(target.X)) ** 2, dim=[0, 1]).item()
        mae_ab += torch.mean(torch.abs(X_mov_transformed - torch.tensor(target.X)), dim=[0, 1]).item()

        transformed_target = transform(target.X, R=rotation_ba_pred, t=translation_ba_pred)
        mse_ba += torch.mean((transformed_target - torch.tensor(src.X)) ** 2, dim=[0, 1]).item()
        mae_ba += torch.mean(torch.abs(transformed_target - torch.tensor(src.X)), dim=[0, 1]).item()

    rotations_ab = np.concatenate(rotations_ab, axis=0)
    translations_ab = np.concatenate(translations_ab, axis=0)
    rotations_ab_pred = np.concatenate(rotations_ab_pred, axis=0)
    translations_ab_pred = np.concatenate(translations_ab_pred, axis=0)

    rotations_ba = np.concatenate(rotations_ba, axis=0)
    translations_ba = np.concatenate(translations_ba, axis=0)
    rotations_ba_pred = np.concatenate(rotations_ba_pred, axis=0)
    translations_ba_pred = np.concatenate(translations_ba_pred, axis=0)

    eulers_ab = np.concatenate(eulers_ab, axis=0)
    eulers_ba = np.concatenate(eulers_ba, axis=0)

    return total_loss * 1.0 / num_examples, total_cycle_loss / num_examples, \
           mse_ab * 1.0 / num_examples, mae_ab * 1.0 / num_examples, \
           mse_ba * 1.0 / num_examples, mae_ba * 1.0 / num_examples, rotations_ab, \
           translations_ab, rotations_ab_pred, translations_ab_pred, rotations_ba, \
           translations_ba, rotations_ba_pred, translations_ba_pred, eulers_ab, eulers_ba
    

# ICP

In [2]:
import open3d as o3d
import numpy as np
from torch.utils.data import Dataset
class TUM(Dataset):
        def __init__(self, size):
                self.pcd1 = o3d.io.read_point_cloud("../4625_after_.pcd")
                self.index = np.random.randint(len(self.pcd1.points),size=size)
                # print(self.index)
                self.points1 = np.array(self.pcd1.points)[self.index] #转为矩阵

                self.pcd2 = o3d.io.read_point_cloud("../4650_after_.pcd")
                self.points2 = np.array(self.pcd2.points)[self.index]

                self.points1 = self.points1[np.newaxis,:]
    
        def __getitem__(self, index):
                points1 = self.points1[index]
                anglex = np.random.uniform() * np.pi / 20
                angley = np.random.uniform() * np.pi / 20
                anglez = np.random.uniform() * np.pi / 20

                cosx = np.cos(anglex)
                cosy = np.cos(angley)
                cosz = np.cos(anglez)
                sinx = np.sin(anglex)
                siny = np.sin(angley)
                sinz = np.sin(anglez)
                Rx = np.array([[1, 0, 0],
                                [0, cosx, -sinx],
                                [0, sinx, cosx]])
                Ry = np.array([[cosy, 0, siny],
                                [0, 1, 0],
                                [-siny, 0, cosy]])
                Rz = np.array([[cosz, -sinz, 0],
                                [sinz, cosz, 0],
                                [0, 0, 1]])
                R_ab = Rx.dot(Ry).dot(Rz)
                R_ba = R_ab.T
                translation_ab = np.array([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5),
                                                np.random.uniform(-0.5, 0.5)])
                translation_ba = -R_ba.dot(translation_ab)


                euler_ab = np.asarray([anglez, angley, anglex])
                euler_ba = -euler_ab[::-1]
                
                return points1.T.astype('float32'), self.points2.T.astype('float32'), R_ab.astype('float32'), \
        translation_ab.astype('float32'), R_ba.astype('float32'), translation_ba.astype('float32'), \
        euler_ab.astype('float32'), euler_ba.astype('float32'), self.index

        def __len__(self):
                return len(self.points1)

In [3]:
pc = TUM(5000)

test_loss, test_cycle_loss, \
test_mse_ab, test_mae_ab, test_mse_ba, test_mae_ba, test_rotations_ab, test_translations_ab, \
test_rotations_ab_pred, \
test_translations_ab_pred, test_rotations_ba, test_translations_ba, test_rotations_ba_pred, \
test_translations_ba_pred, test_eulers_ab, test_eulers_ba = test_one_epoch(pc)
test_rmse_ab = np.sqrt(test_mse_ab)
test_rmse_ba = np.sqrt(test_mse_ba)

test_rotations_ab_pred_euler = npmat2euler(test_rotations_ab_pred)
test_r_mse_ab = np.mean((test_rotations_ab_pred_euler - np.degrees(test_eulers_ab)) ** 2)
test_r_rmse_ab = np.sqrt(test_r_mse_ab)
test_r_mae_ab = np.mean(np.abs(test_rotations_ab_pred_euler - np.degrees(test_eulers_ab)))
test_t_mse_ab = np.mean((test_translations_ab - test_translations_ab_pred) ** 2)
test_t_rmse_ab = np.sqrt(test_t_mse_ab)
test_t_mae_ab = np.mean(np.abs(test_translations_ab - test_translations_ab_pred))

test_rotations_ba_pred_euler = npmat2euler(test_rotations_ba_pred, 'xyz')
test_r_mse_ba = np.mean((test_rotations_ba_pred_euler - np.degrees(test_eulers_ba)) ** 2)
test_r_rmse_ba = np.sqrt(test_r_mse_ba)
test_r_mae_ba = np.mean(np.abs(test_rotations_ba_pred_euler - np.degrees(test_eulers_ba)))
test_t_mse_ba = np.mean((test_translations_ba - test_translations_ba_pred) ** 2)
test_t_rmse_ba = np.sqrt(test_t_mse_ba)
test_t_mae_ba = np.mean(np.abs(test_translations_ba - test_translations_ba_pred))

# filename = 'icp_matrix.txt'
# with open(filename,'w') as f:
#     f.write('==FINAL TEST==\n')
#     f.write('A--------->B\n')
#     f.write('EPOCH:: %d, Loss: %f, Cycle Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
#                     'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f\n'
#                     % (0, test_loss, test_cycle_loss, test_mse_ab, test_rmse_ab, test_mae_ab,
#                         test_r_mse_ab, test_r_rmse_ab,
#                         test_r_mae_ab, test_t_mse_ab, test_t_rmse_ab, test_t_mae_ab))
#     f.write('B--------->A\n')
#     f.write('EPOCH:: %d, Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
#                     'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f\n'
#                     % (0, test_loss, test_mse_ba, test_rmse_ba, test_mae_ba, test_r_mse_ba, test_r_rmse_ba,
#                         test_r_mae_ba, test_t_mse_ba, test_t_rmse_ba, test_t_mae_ba))
print('==FINAL TEST==')
print('A--------->B')
print('EPOCH:: %d, Loss: %f, Cycle Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
                  'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
                  % (-1, test_loss, test_cycle_loss, test_mse_ab, test_rmse_ab, test_mae_ab,
                     test_r_mse_ab, test_r_rmse_ab,
                     test_r_mae_ab, test_t_mse_ab, test_t_rmse_ab, test_t_mae_ab))
print('B--------->A')
print('EPOCH:: %d, Loss: %f, MSE: %f, RMSE: %f, MAE: %f, rot_MSE: %f, rot_RMSE: %f, '
                  'rot_MAE: %f, trans_MSE: %f, trans_RMSE: %f, trans_MAE: %f'
                  % (-1, test_loss, test_mse_ba, test_rmse_ba, test_mae_ba, test_r_mse_ba, test_r_rmse_ba,
                     test_r_mae_ba, test_t_mse_ba, test_t_rmse_ba, test_t_mae_ba))



100%|██████████| 1/1 [00:01<00:00,  1.84s/it]

==FINAL TEST==
A--------->B
EPOCH:: -1, Loss: 0.047889, Cycle Loss: 0.000000, MSE: 0.116747, RMSE: 0.341683, MAE: 0.209915, rot_MSE: 6.312494, rot_RMSE: 2.512468, rot_MAE: 2.207949, trans_MSE: 0.046549, trans_RMSE: 0.215753, trans_MAE: 0.196382
B--------->A
EPOCH:: -1, Loss: 0.047889, MSE: 0.116747, RMSE: 0.341683, MAE: 0.210131, rot_MSE: 6.312494, rot_RMSE: 2.512468, rot_MAE: 2.207949, trans_MSE: 0.048677, trans_RMSE: 0.220628, trans_MAE: 0.191498


# Visualization Sample

In [4]:
src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba, _ = pc[0]
src = PointCloud(src.T, columns=["x", "y", "z"])
target = PointCloud(target.T, columns=["x", "y", "z"])
# Create simpleICP object, add point clouds, and run algorithm!
icp = SimpleICP()
icp.add_point_clouds(target, src)
H, X_mov_transformed, rigid_body_transformation_params = icp.run(max_overlap_distance=1)
visualize_pointcloud(target, X_mov_transformed, .05, R=H[:3,:3], t=H[:3, 3])

Output()

# Export

In [12]:
"""Export to disk"""


def export_mesh_to_obj(path, vertices, faces, vertices2):
    """
    exports mesh as OBJ
    :param path: output path for the OBJ file
    :param vertices: Nx3 vertices
    :param faces: Mx3 faces
    :return: None
    """

    # write vertices starting with "v "
    # write faces starting with "f "

    # ###############
    # DONE: Implement
    v = ""
    f = ""
    v2 = ""
    file = open(path, 'w+')

    if vertices is not None:
        for vertice in vertices:
            v = v + "v "
            for i in vertice:
                v = v + str(i) + " "
            v = v + "104 240 234\n"
            # v = v + "\n"
        
    count = 1
    if faces is not None:
        for face in faces:
            f = f + "f "
            for i in face:
                f = f + str(i+1) + " "
            f = f + "\n"

    if vertices2 is not None:
        for vertice2 in vertices2:
            v2 = v2 + "v "
            for i in vertice2:
                v2 = v2 + str(i) + " "
            v2 = v2 + "252 133 81\n"  
            # v2 = v2 + "\n"    
    file.write(v)
    file.write(v2)
    file.write(f)
    file.close()
        
    # ###############


def export_pointcloud_to_obj(path, pointcloud, pointcloud2=None):
    """
    export pointcloud as OBJ
    :param path: output path for the OBJ file
    :param pointcloud: Nx3 points
    :return: None
    """

    # ###############
    # DONE: Implement
    export_mesh_to_obj(path, pointcloud, None, pointcloud2)
    # ###############

print(rigid_body_transformation_params)
export_pointcloud_to_obj('../icp_test.obj', target.X, X_mov_transformed)


RigidBodyParameters(alpha1=Parameter(initial_value=0.10750457315556647, observed_value=0.0, observation_weight=0.0, estimated_value=0.10750457315556647, estimated_uncertainty=1.4988235443417607e-05, scale_for_logging=57.29577951308232), alpha2=Parameter(initial_value=-0.16308165329963306, observed_value=0.0, observation_weight=0.0, estimated_value=-0.16308165329963306, estimated_uncertainty=1.5435543974382083e-05, scale_for_logging=57.29577951308232), alpha3=Parameter(initial_value=0.011883495138386518, observed_value=0.0, observation_weight=0.0, estimated_value=0.011883495138386518, estimated_uncertainty=2.3124539773152144e-05, scale_for_logging=57.29577951308232), tx=Parameter(initial_value=1.0953829982311267, observed_value=0.0, observation_weight=0.0, estimated_value=1.0953829982311267, estimated_uncertainty=4.9242055285358574e-05, scale_for_logging=1), ty=Parameter(initial_value=0.4325993321892744, observed_value=0.0, observation_weight=0.0, estimated_value=0.4325993321892744, est